<a href="https://colab.research.google.com/github/raphaelpavao/lab-natty-or-not/blob/main/avaliador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORTAÇÕES

In [2]:
import requests
import json
import openai

# KEY

In [3]:
# Configurações da API Google
api_key = ''
# Configurações da API do OpenAI
openai.api_key = ''



#COORDENADAS

In [4]:
#Coordenada geográfica, palavra chave e raio da busca
location = "-22.87404025311588, -47.06161652833972"  # Coordenadas de exemplo (São Paulo)
keyword = "bicicletaria"
radius = 5000

#CHAMADA DA API PARA BUSCAR AS OPÇOES

In [ ]:
url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location}&radius={radius}&keyword={keyword}&key={api_key}"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()

    if 'results' in data and len(data['results']) > 0:
        opcoes = []
        for i in range(min(3, len(data['results']))):
            place_id = data['results'][i].get('place_id', 'N/A')
            name = data['results'][i].get('name', 'N/A')
            opcoes.append((place_id, name))

        opcao1 = opcoes[0] if len(opcoes) > 0 else (None, None)
        opcao2 = opcoes[1] if len(opcoes) > 1 else (None, None)
        opcao3 = opcoes[2] if len(opcoes) > 2 else (None, None)

        print("Digite o número da sua opção:")
        print(f"Opção 1: {opcao1[1]}")
        print(f"opção 2: {opcao2[1]}")
        print(f"opção 3: {opcao3[1]}")
    else:
        print("Nenhum lugar encontrado. Verifique os detalhes da pesquisa.")
else:
    print(f"Erro na requisição: {response.status_code}")


In [6]:
#ESCOLHA DA OPÇÃO

In [ ]:
opcao_escolhida = 2
if opcao_escolhida == 1:
    opcao_selecionada = opcao1[1]
    place_id = opcao1[0]
    print(opcao_selecionada)
    print(place_id)

elif opcao_escolhida == 2:
    opcao_selecionada = opcao2[1]
    place_id = opcao2[0]
    print(opcao_selecionada)
    print(place_id)

elif opcao_escolhida == 3:
    opcao_selecionada = opcao3[1]
    place_id = opcao3[0]
    print(opcao_selecionada)
    print(place_id)

else:
    print("Opção inválida.")





In [7]:
def print_reviews(place_id):
  url_reviews = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&fields=name,place_id,reviews&language=pt&key={api_key}"
  response_reviews = requests.get(url_reviews)
  if response_reviews.status_code == 200:
    data_reviews = response_reviews.json()
    if 'result' in data_reviews and 'reviews' in data_reviews['result']:
      reviews = data_reviews['result']['reviews']
      formatted_reviews = []
      for i, review in enumerate(reviews[:3]):
        formatted_reviews.append(f"Avaliação {i+1}:\nRating: {review['rating']}\nTexto: {review['text']}\nData: {review['time']}\n")
        return '\n'.join(formatted_reviews)
    else:
      return "Nenhuma avaliação encontrada para este lugar."
  else:
    return

In [ ]:
url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&fields=name,place_id,reviews&key={api_key}&language=pt"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()

    if 'result' in data and 'reviews' in data['result']:
        reviews = data['result']['reviews']
        print(f"Nome do Lugar: {data['result']['name']}")
        avaliacoes=""
        for i, review in enumerate(reviews[:5]):
            avaliacoes+= f"*{i+1}:{review['text']}"
        print(avaliacoes)
    else:
        print("Nenhuma avaliação encontrada para este lugar.")
else:
    print(f"Erro na requisição: {response.status_code}")


In [ ]:

# Enviar o texto das avaliações para a API da OpenAI
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "Você é um assistente que ajuda a avaliar textos de reviews de usuários."},
    {"role": "user", "content": f"Avalie as seguintes avaliações. Elas estão separadas por *. Para cada uma delas dê uma nota de 0 a 10 e forneça um resumo dos pontos positivos e negativos. Forneça a nota média e um resumo geral em bullet points.\n\n{avaliacoes}"},
  ],
  max_tokens=500,
  temperature=0.5,
)


# Exibir a resposta da OpenAI
print(response['choices'][0]['message']['content'])

